# Setup your project folder

In [1]:
%cd /path_to/ArcNerf

/group/30042/leoyluo/Immerse/projects/ArcNerf


# Import lib

In [2]:
import os
import time

import numpy as np
import torch
from tqdm import tqdm

from arcnerf.datasets import get_dataset, get_model_feed_in
from arcnerf.geometry.poses import generate_cam_pose_on_sphere, invert_poses
from arcnerf.models import build_model
from arcnerf.render.ray_helper import get_rays
from common.utils.cfgs_utils import load_configs
from common.utils.img_utils import img_to_uint8
from common.utils.logger import Logger
from common.utils.model_io import load_model
from common.utils.torch_utils import torch_to_np
from common.utils.video_utils import write_video

# Specify the model cfgs and model_pt with device

In [3]:
cfgs_file = '/group/30042/leoyluo/Immerse/projects/ArcNerf/configs/inference.yaml'
model_pt = '/group/30042/leoyluo/Immerse/projects/ArcNerf/experiments/capture_qqtiger_nerf/checkpoints/final.pt.tar'
device = 'gpu'  # 'cpu' or 'gpu'

assert os.path.exists(cfgs_file), 'cfgs not exist at {}'.format(cfgs_file)
assert os.path.exists(model_pt), 'model file not exist at {}'.format(model_pt)

# Set up cfgs, device, models

In [4]:
cfgs = load_configs(cfgs_file)
logger = Logger()

if torch.cuda.is_available() and device == 'gpu':
    torch.cuda.set_device(0)
    
model = build_model(cfgs, None)
model = load_model(logger, model, None, model_pt, cfgs)
if device == 'gpu':
    model.cuda()

2022-05-20 11:28:20.076 | INFO     | common.utils.logger:add_log:34 - Successfully loaded checkpoint from /group/30042/leoyluo/Immerse/projects/ArcNerf/experiments/Finish/Capture/capture_qqtiger_neus_nerfpprgb/checkpoints/final.pt.tar (at epoch 300000)... Keep Train: True(Start from 300000)


# Get intrinsic from data or set manually

In [5]:
# get from dataset if you don't know intrinsic
# dataset = get_dataset(cfgs.dataset, cfgs.dir.data_dir, mode='eval', logger=logger)
# intrinsic = dataset.get_intrinsic(torch_tensor=False)
# wh = dataset.get_wh()

# set it manually
focal = 800.0
wh = (540, 960)
intrinsic = np.array([[focal, 0, wh[0]/2.0],
                      [0, focal, wh[1]/2.0],
                      [0, 0, 1]])  # (3, 3)
dtype = torch.float32

# Get visual camera position

In [6]:
# set up camera path params
mode = 'circle'      # render cam type
n_cam = 20           # render num of cam
repeat = 3         # repeat the render video
radius = 2.0         # sphere radius
u_start = 0.75       # for start pos u in (0~1)
v_ratio = -0.2       # for circle path, vertical position
fps = 5              # render video fps

# get camera pose
c2w = generate_cam_pose_on_sphere(
    mode,
    radius,
    n_cam,
    u_start=u_start,
    v_ratio=v_ratio,
    close=True
)
logger.add_log('Num of camera in the path {}'.format(n_cam))

# get rays
logger.add_log('Getting rays...')
inputs = []
for cam_id in range(n_cam):
    t_intrinsic = torch.tensor(intrinsic, dtype=dtype)
    t_c2w = torch.tensor(c2w[cam_id], dtype=dtype)
    if device == 'gpu':  # create tensor on gpu
        t_intrinsic = t_intrinsic.cuda()
        t_c2w = t_c2w.cuda()
    ray_bundle = get_rays(wh[0], wh[1], t_intrinsic, t_c2w, wh_order=False)  # (HW, 3) * 2
    input = {
        'c2w': c2w[cam_id],   # (4, 4) np array
        'intrinsic': intrinsic,  # (3, 3) np array
        'rays_o': ray_bundle[0][None, :],  # (1, HW, 3) torch tensor
        'rays_d': ray_bundle[1][None, :],  # (1, HW, 3) torch tensor
        'rays_r': ray_bundle[3][None, :]   # (1, HW, 1) torch tensor
    }
    inputs.append(input)
logger.add_log('Each camera has num of rays {}'.format(inputs[0]['rays_o'].shape[1]))

2022-05-20 11:28:23.735 | INFO     | common.utils.logger:add_log:34 - Num of camera in the path 20
2022-05-20 11:28:23.736 | INFO     | common.utils.logger:add_log:34 - Getting rays...
2022-05-20 11:28:27.678 | INFO     | common.utils.logger:add_log:34 - Each camera has num of rays 518400


# Set up surface_render params

In [7]:
chunk_rays_factor = 1            # set this to allow model to process more rays in surface_render mode
method = 'secant_root_finding'   # method to find surface pts, ['sphere_tracing', 'secant_root_finding']
level = 50.0                     # level set for mesh extracting. For sigma is around 50(No accurate). For sdf is 0.
grad_dir = 'descent'             # if 'descent', sigma is larger than level in obj(NeRF), if 'ascent' is smaller(SDF)

# model reset chunk_rays
origin_chunk_rays = model.get_chunk_rays()
model.set_chunk_rays(origin_chunk_rays * chunk_rays_factor)

# Run surface rendering

In [9]:
images = []
total_forward_time = 0.0
for rays in tqdm(inputs):
    # empty cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    feed_in, batch_size = get_model_feed_in(rays, device)  # only read rays_o/d here, (1, WH, 3)
    assert batch_size == 1, 'Only one image is sent to model at once for inference...'

    time0 = time.time()
    output = model.surface_render(feed_in, method=method, level=level, grad_dir=grad_dir)  # call surface rendering
    total_forward_time += (time.time() - time0)

    # get rgb only
    rgb = output['rgb']  # (1, HW, 3)
    rgb = img_to_uint8(torch_to_np(rgb).copy()).reshape(int(wh[1]), int(wh[0]), 3)  # (H, W, 3), bgr
    images.append(rgb)

# log time
logger.add_log(
    '   Surface Render {} image, each hw({}/{}) total time {:.2f}s'.format(
        len(images), wh[1], wh[0], total_forward_time
    )
)
logger.add_log('    Each image takes time {:.2f}s'.format(total_forward_time / float(len(images))))

100%|██████████| 20/20 [01:50<00:00,  5.51s/it]
2022-05-20 11:30:17.833 | INFO     | common.utils.logger:add_log:34 -    Surface Render 20 image, each hw(960/540) total time 106.42s
2022-05-20 11:30:17.834 | INFO     | common.utils.logger:add_log:34 -     Each image takes time 5.32s


# Write down video, view it locally

In [15]:
file_path = './results/surface_render.mp4'
write_video(images * repeat, file_path, fps=fps)
logger.add_log('Write surface render videos to {}'.format(file_path))

2022-05-20 11:33:39.304 | INFO     | common.utils.logger:add_log:34 - Write surface render videos to ./results/surface_render.mp4
